In [2]:
import mfem.ser as mfem
import numpy as np

In [3]:
class EDependentCoefficient(mfem.PyCoefficient):
    """Coefficient class for energy-dependent values.

    This class provides a piecewise-constant coefficient function based on energy groups.
    The energy range is defined from E_start (higher energy) to E_end (lower energy),
    with energy values sorted in descending order. The corresponding energy-dependent
    values in data are also arranged from higher to lower energy. This ordering is based
    on the logic of the Continuous Slowing Down Approximation (CSDA) approach, where the energy
    loss of charged particles is treated as a continuous process, naturally leading to a descending
    energy order.

    Attributes:
        E_start (float): Upper bound of the energy range.
        E_end (float): Lower bound of the energy range.
        data (array-like or float): Energy-dependent values per energy group,
            or a constant value.

    Args:
        data (float or list of float): Energy-dependent data (constant or group-based).
        E_start (float): Start energy for energy group mapping.
        E_end (float): End energy for energy group mapping.

    Examples:
        Constant energy-dependent value:
            EDependentCoefficient(1.0, E_start=1.0, E_end=0.01)

        Energy-dependent coefficient:
            EDependentCoefficient(data, E_start=1.0, E_end=0.01)
    """
    def __init__(self, data, E_start, E_end):
        super(EDependentCoefficient, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.n_groups = len(data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        """Evaluate the coefficient value at the given energy point.

        Args:
            x (list or array-like): Input point, where x[1] is the energy value.

        Returns:
            float: Corresponding coefficient value.
        """
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.data[i])
        return float(self.data[-1])


In [4]:
class XDependentCoefficient(mfem.PyCoefficient):
    """Coefficient for the spatial-dependent values.

    This class provides a piecewise-constant coefficient function based on spatial cell number.
    The spatial range is defined from x_start (lower bound) to x_end (upper bound),
    with spatial values sorted in ascending order. The corresponding spatial-dependent
    values in data are also arranged from lower to upper according to their grouping.
    
    Attributes:
        x_start (float): Lower bound of the spatial range.
        x_end (float): Upper bound of the spatial range.
        data (array-like or float): Spatial-dependent values per cell, or a constant value.

    Args:
        data (float or list of float): Spatial-dependent data (constant or cell-based).
        x_start (float): Start value for spatial mapping.
        x_end (float): End value for spatial mapping.

    Examples:
        Constant spatial-dependent value:
            XDependentCoefficient(1.0, x_start=0.0, x_end=10.0)

        Spatial-dependent coefficient:
            XDependentCoefficient(data, x_start=0.0, x_end=10.0)
    """
    def __init__(self, data, x_start, x_end):
        super(XDependentCoefficient, self).__init__()
        self.x_start = x_start
        self.x_end = x_end
        if isinstance(data, (int, float)):
            self.constant = True
            self.constant_value = float(data)
        else:
            self.constant = False
            self.data = data
            self.n_groups = len(data)
            self.x_bins = np.linspace(x_start, x_end, self.n_groups + 1)

    def EvalValue(self, x):
        """Evaluate the spatial-dependent value at the given spatial point.

        Args:
            x (list or array-like): Input point, where x[0] is the spatial value.

        Returns:
            float: Corresponding spatial-dependent value.
        """
        if self.constant:
            return self.constant_value

        x_val = x[0]
        for i in range(self.n_groups - 1):
            if np.isclose(x_val, self.x_bins[i+1]):
                return float(self.data[i+1])
            if self.x_bins[i] <= x_val < self.x_bins[i+1]:
                return float(self.data[i])
        return float(self.data[-1])


In [5]:
class TotalXSCoefficientE(mfem.PyCoefficient):
    """Coefficient class for the total macroscopic cross-section Σ_t(E).

    This class provides a piecewise-constant coefficient function based on energy groups.
    The energy range is defined from E_start (higher energy) to E_end (lower energy),
    with energy values sorted in descending order. The corresponding total cross-section
    values in xs_t_data are also arranged from higher to lower energy. This ordering is based
    on the logic of the Continuous Slowing Down Approximation (CSDA) approach, where the energy
    loss of charged particles is treated as a continuous process, naturally leading to a descending
    energy order.

    Attributes:
        E_start (float): Upper bound of the energy range.
        E_end (float): Lower bound of the energy range.
        xs_t_data (array-like or float): Total cross-section values per energy group,
            or a constant value.

    Args:
        xs_t_data (float or list of float): Total cross-section data (constant or group-based).
        E_start (float): Start energy for energy group mapping.
        E_end (float): End energy for energy group mapping.

    Examples:
        Constant cross-section:
            TotalXSCoefficient(1.0, E_start=1.0, E_end=0.01)

        Energy-dependent cross-section:
            TotalXSCoefficient(xs_t_data, E_start=1.0, E_end=0.01)
    """

    def __init__(self, xs_t_data, E_start, E_end):
        super(TotalXSCoefficientE, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_t_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_t_data)
        else:
            self.constant = False
            self.xs_t_data = xs_t_data
            self.n_groups = len(xs_t_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        """Evaluate the total cross-section at the given energy point.

        Args:
            x (list or array-like): Input point, where x[1] is the energy value.

        Returns:
            float: Corresponding total cross-section value.
        """
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_t_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_t_data[i])
        return float(self.xs_t_data[-1])

In [6]:
class ScatteringXSCoefficientE(mfem.PyCoefficient):
    """Coefficient class for the scattering cross-section Σ_s(E).

    This class provides a piecewise-constant coefficient function based on energy groups.
    The energy range is defined from E_start (higher energy) to E_end (lower energy),
    with energy values sorted in descending order. The corresponding scattering cross-section
    values in xs_s_data are also arranged from higher to lower energy. This ordering is based
    on the logic of the Continuous Slowing Down Approximation (CSDA) approach, where the energy
    loss of charged particles is treated as a continuous process, naturally leading to a descending
    energy order.

    Attributes:
        E_start (float): Upper bound of the energy range.
        E_end (float): Lower bound of the energy range.
        xs_s_data (array-like or float): Scattering cross-section values per energy group,
            or a constant value.

    Args:
        xs_s_data (float or list of float): Scattering cross-section data (constant or group-based).
        E_start (float): Start energy for energy group mapping.
        E_end (float): End energy for energy group mapping.

    Examples:
        Constant cross-section:
            ScatteringXSCoefficient(1.0, E_start=1.0, E_end=0.01)

        Energy-dependent cross-section:
            ScatteringXSCoefficient(xs_s_data, E_start=1.0, E_end=0.01)
    """

    def __init__(self, xs_s_data, E_start, E_end):
        super(ScatteringXSCoefficientE, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(xs_s_data, (int, float)):
            self.constant = True
            self.constant_value = float(xs_s_data)
        else:
            self.constant = False
            self.xs_s_data = xs_s_data
            self.n_groups = len(xs_s_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        """Evaluate the scattering cross-section at the given energy point.

        Args:
            x (list or array-like): Input point, where x[1] is the energy value.

        Returns:
            float: Corresponding scattering cross-section value.
        """
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.xs_s_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.xs_s_data[i])
        return float(self.xs_s_data[-1])

In [7]:
class StoppingPowerCoefficientE(mfem.PyCoefficient):
    """Coefficient class for the stopping power S(E).

    This class provides a piecewise-constant coefficient function based on energy groups.
    The energy range is defined from E_start (higher energy) to E_end (lower energy),
    with energy values sorted in descending order. The corresponding stopping power
    values in S_data are also arranged from higher to lower energy. This ordering is based
    on the logic of the Continuous Slowing Down Approximation (CSDA) approach, where the energy
    loss of charged particles is treated as a continuous process, naturally leading to a descending
    energy order.

    Attributes:
        E_start (float): Upper bound of the energy range.
        E_end (float): Lower bound of the energy range.
        S_data (array-like or float): Stopping power values per energy group,
            or a constant value.

    Args:
        S_data (float or list of float): Stopping power data (constant or group-based).
        E_start (float): Start energy for energy group mapping.
        E_end (float): End energy for energy group mapping.

    Examples:
        Constant cross-section:
            TotalXSCoefficient(1.0, E_start=1.0, E_end=0.01)

        Energy-dependent cross-section:
            TotalXSCoefficient(xs_t_data, E_start=1.0, E_end=0.01)
    """
    def __init__(self, S_data, E_start, E_end):
        super(StoppingPowerCoefficientE, self).__init__()
        self.E_start = E_start
        self.E_end = E_end
        if isinstance(S_data, (int, float)):
            self.constant = True
            self.constant_value = float(S_data)
        else:
            self.constant = False
            self.S_data = S_data
            self.n_groups = len(S_data)
            self.E_bins = np.linspace(E_start, E_end, self.n_groups + 1)

    def EvalValue(self, x):
        """Evaluate the total cross-section at the given energy point.

        Args:
            x (list or array-like): Input point, where x[1] is the energy value.

        Returns:
            float: Corresponding total cross-section value.
        """
        if self.constant:
            return self.constant_value
        E = x[1]
        for i in range(self.n_groups - 1):
            if np.isclose(E, self.E_bins[i+1]):
                return float(self.S_data[i+1])
            if E <= self.E_bins[i] and E > self.E_bins[i+1]:
                return float(self.S_data[i])
        return float(self.S_data[-1])

In [8]:
class InflowCoefficient(mfem.PyCoefficient):
    """Coefficient class for the inflow boundary condition.

    This class represents a coefficient for the inflow boundary condition.
    The coefficient returns a specified inflow value based on the angular direction `mu` and
    optional boundary flags. If `xl` is set to True, the coefficient returns the inflow value
    only when `mu < 0` (left boundary). If `xr` is set to True, the coefficient returns the
    inflow value only when `mu > 0` (right boundary). If both flags are True or if neither flag
    is provided (default), the inflow value is returned unconditionally.

    Args:
        inflow (float): The inflow value to be used.
        mu (float): The angular direction (e.g., cosine of the angle) used to determine if the 
            inflow condition is met.
        xl (bool, optional): If True, apply the inflow condition on the left boundary (when mu > 0).
            Defaults to None.
        xr (bool, optional): If True, apply the inflow condition on the right boundary (when mu < 0).
            Defaults to None.
    """

    def __init__(self, inflow, mu, xl=None, xr=None):
        super(InflowCoefficient, self).__init__()
        self.inflow = inflow
        self.mu = mu
        self.xl = xl
        self.xr = xr

    def EvalValue(self, x):
        """Evaluate the inflow coefficient.

        Depending on the specified boundary flags, this method returns the inflow value only if
        the corresponding condition based on `mu` is satisfied. If neither or both boundary flags
        are provided, the inflow value is returned unconditionally.

        Args:
            x (list or array-like): Input point.

        Returns:
            float: The inflow value if the condition is met, otherwise 0.0.
        """

        if self.xl is True and self.xr is not True:
            if self.mu > 0:
                return self.inflow
            else:
                return 0.0 
        
        elif self.xr is True and self.xl is not True:
            if self.mu < 0:
                return self.inflow
            else:
                return 0.0

        else:
            return self.inflow


In [15]:
def test_TotalXSCoefficientE():
    xs_t_data = [0,1,2,3,4]
    coeff = TotalXSCoefficientE(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_TotalXSCoefficientE_Constant():
    coeff = TotalXSCoefficientE(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

print("Test for TotalXSCoefficientE")
test_TotalXSCoefficientE()
print("Test for TotalXSCoefficientE Constant")
test_TotalXSCoefficientE_Constant()

def test_ScatteringXSCoefficientE():
    xs_t_data = [0,1,2,3,4]
    coeff = ScatteringXSCoefficientE(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_ScatteringXSCoefficientE_Constant():
    coeff = ScatteringXSCoefficientE(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

print("Test for ScatteringXSCoefficientE")
test_ScatteringXSCoefficientE()
print("Test for ScatteringXSCoefficientE Constant")
test_ScatteringXSCoefficientE_Constant()

def test_StoppingPowerCoefficientE():
    xs_t_data = [0,1,2,3,4]
    coeff = StoppingPowerCoefficientE(xs_t_data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_StoppingPowerCoefficientE_Constant():
    coeff = StoppingPowerCoefficientE(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

print("Test for StoppingPowerCoefficientE")
test_StoppingPowerCoefficientE()
print("Test for StoppingPowerCoefficientE Constant")
test_StoppingPowerCoefficientE_Constant()

def test_EDependentCoefficient():
    data = [0,1,2,3,4]
    coeff = EDependentCoefficient(data, 1, 0.01)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([0.0, 0.604]))
    print(coeff.EvalValue([0.0, 0.406]))
    print(coeff.EvalValue([0.0, 0.208]))
    print(coeff.EvalValue([0.0, 0.01]))

def test_EDependentCoefficient_Constant():
    coeff = EDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([0.0, 1]))
    print(coeff.EvalValue([0.0, 0.802]))
    print(coeff.EvalValue([1.0, 0.604]))
    print(coeff.EvalValue([0.3, 0.406]))
    print(coeff.EvalValue([0.6, 0.208]))
    print(coeff.EvalValue([0.8, 0.01]))

print("Test for EDependentCoefficient")
test_EDependentCoefficient()
print("Test for EDependentCoefficient Constant")
test_EDependentCoefficient_Constant()


def test_XDependentCoefficient():
    data = [0,1,2,3,4]
    coeff = XDependentCoefficient(data, 0.0, 1.0)
    print(coeff.EvalValue([0.01, 0.0]))
    print(coeff.EvalValue([0.208, 0.0]))
    print(coeff.EvalValue([0.406, 0.0]))
    print(coeff.EvalValue([0.604, 0.0]))
    print(coeff.EvalValue([0.802, 0.0]))
    print(coeff.EvalValue([1.0, 0.0]))

def test_XDependentCoefficient_Constant():
    coeff = XDependentCoefficient(40, 0, 1)
    print(coeff.EvalValue([1, 0.0]))
    print(coeff.EvalValue([0.802, 0.0]))
    print(coeff.EvalValue([0.604, 0.0]))
    print(coeff.EvalValue([0.406, 0.0]))
    print(coeff.EvalValue([0.208, 0.0]))
    print(coeff.EvalValue([0.01, 0.0]))

print("Test for XDependentCoefficient")
test_XDependentCoefficient()
print("Test for XDependentCoefficient Constant")
test_XDependentCoefficient_Constant()

def test_inflow_coefficient():
    """Run tests for InflowCoefficientSN with different configurations."""
    dummy_x = [0, 0]  # Dummy input; x is not used in EvalValue
    
    # Case 1: Both XR and xl are None (default) -> always return inflow
    coeff = InflowCoefficient(100, mu=0.5, xr=None, xl=None)
    assert coeff.EvalValue(dummy_x) == 100, "Test case 1 failed (positive mu, no flags)"
    coeff = InflowCoefficient(100, mu=-0.5, xr=None, xl=None)
    assert coeff.EvalValue(dummy_x) == 100, "Test case 1 failed (negative mu, no flags)"
    
    # Case 2: XR=True, xl=None -> return inflow only when mu > 0
    coeff = InflowCoefficient(200, mu=0.5, xr=True, xl=None)
    assert coeff.EvalValue(dummy_x) == 0.0, "Test case 2 failed (positive mu, XR flag)"
    coeff = InflowCoefficient(200, mu=-0.5, xr=True, xl=None)
    assert coeff.EvalValue(dummy_x) == 200.0, "Test case 2 failed (negative mu, XR flag)"
    
    # Case 3: xl=True, XR=None -> return inflow only when mu < 0
    coeff = InflowCoefficient(300, mu=-0.5, xr=None, xl=True)
    assert coeff.EvalValue(dummy_x) == 0.0, "Test case 3 failed (negative mu, xl flag)"
    coeff = InflowCoefficient(300, mu=0.5, xr=None, xl=True)
    assert coeff.EvalValue(dummy_x) == 300, "Test case 3 failed (positive mu, xl flag)"
    
    # Case 4: Both XR and xl are True -> return inflow unconditionally
    coeff = InflowCoefficient(400, mu=0.5, xr=True, xl=True)
    assert coeff.EvalValue(dummy_x) == 400, "Test case 4 failed (positive mu, both flags)"
    coeff = InflowCoefficient(400, mu=-0.5, xr=True, xl=True)
    assert coeff.EvalValue(dummy_x) == 400, "Test case 4 failed (negative mu, both flags)"
    
    print("All tests passed!")
if __name__ == "__main__":
    test_inflow_coefficient()


Test for TotalXSCoefficientE
0.0
1.0
2.0
3.0
4.0
4.0
Test for TotalXSCoefficientE Constant
40.0
40.0
40.0
40.0
40.0
40.0
Test for ScatteringXSCoefficientE
0.0
1.0
2.0
3.0
4.0
4.0
Test for ScatteringXSCoefficientE Constant
40.0
40.0
40.0
40.0
40.0
40.0
Test for StoppingPowerCoefficientE
0.0
1.0
2.0
3.0
4.0
4.0
Test for StoppingPowerCoefficientE Constant
40.0
40.0
40.0
40.0
40.0
40.0
Test for EDependentCoefficient
0.0
1.0
2.0
3.0
4.0
4.0
Test for EDependentCoefficient Constant
40.0
40.0
40.0
40.0
40.0
40.0
Test for XDependentCoefficient
0.0
1.0
2.0
3.0
4.0
4.0
Test for XDependentCoefficient Constant
40.0
40.0
40.0
40.0
40.0
40.0
All tests passed!


In [10]:
class VectorConstCoefficient(mfem.VectorConstantCoefficient):
    """Coefficient for a vector-valued function composed of two velocity components for 2D FE calculations.

    This class computes a 2-dimensional vector coefficient by combining two scalar coefficients:
    one for the first velocity component and one for the second velocity component. The individual
    scalar coefficients are evaluated at a given input point x, and their values are assembled into
    a two-element vector.

    Args:
        vel1_coeff (mfem.PyCoefficient): Coefficient representing the first velocity component.
        vel2_coeff (mfem.PyCoefficient): Coefficient representing the second velocity component.

    Attributes:
        vel1_coeff (mfem.PyCoefficient): The coefficient for the first velocity component.
        vel2_coeff (mfem.PyCoefficient): The coefficient for the second velocity component.

    Examples:
        vector_coeff = VectorConstCoefficient(vel1_coeff, vel2_coeff)
        value = vector_coeff.EvalValue(x)
        print(value)  # Output: [vel1_val, vel2_val]
    """

    def __init__(self, vel1_coeff, vel2_coeff):
        super(VectorConstCoefficient, self).__init__(2)
        self.vel1_coeff = vel1_coeff 
        self.vel2_coeff = vel2_coeff

    def EvalValue(self, x):
        """Evaluate the vector coefficient at the given input point.

        This method returns a two-element list [vel1_val, vel2_val] by evaluating the respective
        scalar coefficients at the point x using their EvalValue methods.
        """
        vel1_val = self.vel1_coeff.EvalValue(x) 
        vel2_val = self.vel2_coeff.EvalValue(x) 
        return [vel1_val, vel2_val]


In [11]:
# ---------------------- Test Code ----------------------
def test_VectorCoefficient():
    """Test for VectorCoefficient using dummy scalar coefficients."""
    
    # Create a dummy coefficient class that mimics mfem.PyCoefficient
    class DummyCoefficient(mfem.PyCoefficient):
        def __init__(self, value):
            self.value = value

        def EvalValue(self, x):
            return self.value

    # Dummy input point (the content of x is irrelevant for the dummy coefficients)
    dummy_x = [0, 0]

    # Instantiate dummy coefficients with constant values
    vel1 = DummyCoefficient(1.23)
    vel2 = DummyCoefficient(4.56)

    # Create an instance of VectorCoefficient using the dummy coefficients
    vector_coeff = VectorCoefficient(vel1, vel2)

    # Evaluate the vector coefficient at the dummy point
    try:
        result = vector_coeff.EvalValue(dummy_x)
    except AttributeError as e:
        print("Test failed: AttributeError encountered. Check for typos in attribute names.", e)
        return

    # Expected result is a list with the two dummy values
    expected = [1.23, 4.56]
    assert result == expected, f"Expected {expected} but got {result}"

    print("Test passed: VectorCoefficient returns expected vector.")

if __name__ == '__main__':
    test_VectorCoefficient()


NameError: name 'VectorCoefficient' is not defined

In [12]:
class VectorConstCoefficient(mfem.VectorConstantCoefficient):
    """Coefficient for a vector-valued function representing constant velocity values for 1D, 2D, or 3D computations.

    This class computes a vector coefficient by combining up to three scalar values:
    one for the x-direction velocity (required), one for the y-direction velocity (optional),
    and one for the z-direction velocity (optional). If only the first velocity value is provided,
    the coefficient is treated as 1-dimensional. If both the first and second velocity values are provided,
    it is treated as 2-dimensional. If all three are provided, the coefficient is 3-dimensional.

    Args:
        vel1_val (float): Constant velocity value for the x-direction (required).
        vel2_val (float, optional): Constant velocity value for the y-direction. If not provided, the coefficient is 1D.
        vel3_val (float, optional): Constant velocity value for the z-direction. If not provided and vel2_val is provided,
            the coefficient is 2D; if provided, the coefficient is 3D.

    Examples:
        >>> coeff1 = VectorConstCoefficient(3.0)
        >>> print(coeff1.EvalValue([0]))  # Output: [3.0]
        >>> coeff2 = VectorConstCoefficient(3.0, 4.0)
        >>> print(coeff2.EvalValue([0]))  # Output: [3.0, 4.0]
        >>> coeff3 = VectorConstCoefficient(3.0, 4.0, 5.0)
        >>> print(coeff3.EvalValue([0]))  # Output: [3.0, 4.0, 5.0]
    """
    def __init__(self, vel1_val, vel2_val=None, vel3_val=None):
        if vel2_val is None and vel3_val is None:
            dimension = 1
        elif vel3_val is None:
            dimension = 2
        else:
            dimension = 3
        super(VectorConstCoefficient, self).__init__(dimension)
        self.dim = dimension
        self.vel1_val = vel1_val
        self.vel2_val = vel2_val
        self.vel3_val = vel3_val

    def EvalValue(self, x):
        """Evaluate the constant vector coefficient at the given input point.

        This method returns a vector of constant velocity values whose dimension depends on the
        number of input values provided during initialization. The input point x is not used in the
        evaluation.

        Args:
            x (list or array-like): Input point (unused).

        Returns:
            list: A list containing the constant velocity values.
        """
        if self.dim == 1:
            return [self.vel1_val]
        elif self.dim == 2:
            return [self.vel1_val, self.vel2_val]
        else:
            return [self.vel1_val, self.vel2_val, self.vel3_val]


In [13]:
def test_VectorConstCoefficient():
    """Test for the VectorConstCoefficient class in 1D, 2D, and 3D cases."""
    dummy_x = [0]  # Dummy input point (unused)

    # Test 1D: Only vel1_val provided.
    coeff1 = VectorConstCoefficient(3.0)
    result1 = coeff1.EvalValue(dummy_x)
    assert result1 == [3.0], f"1D test failed: expected [3.0], got {result1}"

    # Test 2D: vel1_val and vel2_val provided.
    coeff2 = VectorConstCoefficient(3.0, 4.0)
    result2 = coeff2.EvalValue(dummy_x)
    assert result2 == [3.0, 4.0], f"2D test failed: expected [3.0, 4.0], got {result2}"

    # Test 3D: vel1_val, vel2_val, and vel3_val provided.
    coeff3 = VectorConstCoefficient(3.0, 4.0, 5.0)
    result3 = coeff3.EvalValue(dummy_x)
    assert result3 == [3.0, 4.0, 5.0], f"3D test failed: expected [3.0, 4.0, 5.0], got {result3}"

    print("All tests passed for VectorConstCoefficient!")


if __name__ == "__main__":
    test_VectorConstCoefficient()


All tests passed for VectorConstCoefficient!
